In [1]:
%cd .. 

/Users/nguyenhuyhai/20221/lab_blockchain/orchai_validator


In [2]:
from labeling.ETLProcessor import ETLProcessor
from labeling.tools import get_spark, query
from omegaconf import OmegaConf

In [3]:
config = OmegaConf.load("config/local.yaml")
spark = get_spark('lib/postgresql-42.5.0.jar')

df = query(spark, **config.src)
df.printSchema()

22/12/14 14:20:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/14 14:20:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Successfully queried data from database
root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: boolean (nullable = true)
 |-- vote: boolean (nullable = true)



In [4]:
accept_rate = 0.1
concentration_level = 0.9
vote_score = 2
propose_score = 6
A = 9
B = 4
C = 2
D = 4

"""
suitable parameters are:
vote_proposed_win_size = 14400 (blocks) 
combine_win_size = 150 (blocks)
label_win_size = 100800 (blocks)
Approximate each block ~~ 6seconds
"""
start_block = 7100500
end_block = 7100599
top_validators = 50


vote_proposed_win_size = 500 #This is just test, favourable is
combine_win_size = 10 
label_win_size = 60


df = ETLProcessor.data_scoring(df, accept_rate, concentration_level, vote_score, propose_score, A, B, C, D, start_block, end_block, top_validators ,vote_proposed_win_size, combine_win_size, label_win_size)


------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------


------------------------------------------------
Sucessfully filter data
------------------------------------------------
------------------------------------------------
Sucessfully combine data
------------------------------------------------
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [5]:
df.show(200)

+--------------------+--------------------+------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------------------+------------------+
|    operator_address|     comission_score|             score|voting_power_score|tokens_proportion|     commission_rate|         self_bonded|   self_bonded_score|    delegator_shares|           tokens|block_height|vote_propose_score|             label|
+--------------------+--------------------+------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------------------+------------------+
|oraivaloper17s4d9...|1.569325058792259E-8|               0.0|               1.0|            0E-10| 0.09999999843067496|79836806970414980...|                 0.0|10503061126957536...|           2.0000|     7100499|               0.0|        

In [6]:
import pyspark.sql.functions as F


In [ ]:
top = df.filter(
        df.block_height.between(start_block,end_block)).\
        groupBy("operator_address").agg({"score": "mean"}).\
        orderBy(F.col("avg(score)").desc())

list = []
for i in top.head(top_validators):
    list.append(i.operator_address)

df = df.filter(
    df.operator_address.isin(list)
)